## Preparing the environment

In [1]:
import numpy as np

import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

from datasets import load_dataset
import transformers

from tqdm import tqdm

import matplotlib.pyplot as plt



device = "cuda" if torch.cuda.is_available() else "cpu"
data_path = "../data"

In [2]:
seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = False

## Loading dataset

In [3]:
train = load_dataset("EleutherAI/pile-deduped-pythia-random-sampled", cache_dir=data_path, split="train")
train = train.with_format("torch")

## Training

### Full-rank Training

In [4]:
batch_size = 256
global_batch_size = 512
accumulation_steps = global_batch_size / batch_size

learning_rate = 1e-4
betas = (0.9, 0.95)
eps = 1e-8
gradient_clipping = 1.0
weight_decay = 0.1

warmup_iters = 256

train_iters = 2048

In [5]:
train_loader = iter(DataLoader(train, batch_size=batch_size, shuffle=False))

In [6]:
model = transformers.AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-14m", revision="step1").to(device)

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=betas, eps=eps, weight_decay=weight_decay)

scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_iters, num_training_steps=train_iters)

In [8]:
model.train()

for iter in tqdm(range(train_iters)):
    optimizer.zero_grad()

    if iter > train_iters:
        break
    x = next(train_loader)["Tokens"].to('cuda')

    output = model(input_ids=x, labels=x)
    loss = output.loss
    loss.backward()

    if (iter + 1) % accumulation_steps == 0 or iter + 1 == train_iters: 
        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clipping)
        optimizer.step()
        scheduler.step()
    
    if iter % 128 == 0:
        print(loss.item())

  0%|          | 1/2048 [00:00<17:39,  1.93it/s]

11.03548526763916


  6%|▋         | 131/2048 [00:20<04:09,  7.69it/s]

10.8258056640625


 13%|█▎        | 257/2048 [00:41<06:45,  4.42it/s]

...

 88%|████████▊ | 1795/2048 [04:44<00:39,  6.42it/s]

8.974088668823242


 94%|█████████▍| 1921/2048 [05:05<00:26,  4.84it/s]

9.066380500793457


100%|██████████| 2048/2048 [05:26<00:00,  6.27it/s]
